In [13]:
# WebScrape MLB_Draft
## Pre-processing and cleaning

In [2]:
import os
import urllib
import webbrowser
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

%matplotlib inline

In [13]:
# The url we will be scraping
url_2016 = "http://www.baseball-reference.com/draft/?overall_pick=1&draft_type=junreg&"

# get the html
html = urllib.request.urlopen(url_2016)

# create the BeautifulSoup object
soup = BeautifulSoup(html, "lxml")

In [14]:
# Scraping Column headers
# Extract the necessary values for the column headers from the table
# and store them as a list
column_headers = [th.getText() for th in soup.findAll('th', limit=25)]
column_headers = [s for s in column_headers if len(s) != 0]
column_headers = column_headers[1:23]
column_headers

['Rnd',
 'DT',
 'FrRnd',
 'RdPck',
 'Tm',
 'Signed',
 'Name',
 'Pos',
 'WAR',
 'G',
 'AB',
 'HR',
 'BA',
 'OPS',
 'G',
 'W',
 'L',
 'ERA',
 'WHIP',
 'SV',
 'Type',
 'Drafted Out of']

In [15]:
len(column_headers)

22

In [16]:
# The data is found within the table rows
# We want the elements from the 5th row and on
table_rows = soup.find_all("tr")[1:]
table_rows[5]

<tr class=""><td>2016</td><td align="right"><a href="/draft/index.cgi?year_ID=2016&amp;draft_round=1&amp;draft_type=junreg&amp;query_type=year_round">1</a></td><td align="right"></td><td align="right"><a href="/draft/index.cgi?team_ID=PHI&amp;draft_round=1&amp;draft_type=junreg&amp;query_type=franch_round">FrRnd</a></td><td align="right">1</td><td><a href="/draft/index.cgi?team_ID=PHI&amp;year_ID=2016&amp;draft_type=junreg&amp;query_type=franch_year">Phillies</a></td><td align="center" csk="1">Y</td><td>Mickey Moniak (<a href="/register/player.cgi?id=moniak000mic">minors</a>)</td><td><a href="/draft/index.cgi?pos=OF&amp;year_ID=2016&amp;draft_type=junreg&amp;query_type=pos_year">OF</a></td><td align="right" csk="-1000"></td><td align="right"></td><td align="right"></td><td align="right"></td><td align="right"></td><td align="right"></td><td align="right"></td><td align="right"></td><td align="right"></td><td align="right"></td><td align="right"></td><td align="right"></td><td align="">

In [17]:
def extract_player_data(table_rows):
    """
    Extract and return the the desired information from the td elements within
    the table rows.
    """
    # create the empty list to store the player data
    player_data = []
    
    for row in table_rows:  # for each row do the following

        # Get the text for each table data (td) element in the row
        # Some player names end with ' HOF', if they do, get the text excluding
        # those last 4 characters,
        # otherwise get all the text data from the table data
        player_list = [td.get_text() for td in row.find_all("td")]
        
        
        # there are some empty table rows, which are the repeated 
        # column headers in the table
        # we skip over those rows and and continue the for loop
        if not player_list:
            continue
            
        # Now append the data to list of data
        player_data.append(player_list)
        
    return player_data

In [18]:
# Create datafram with 2016 draft data

In [19]:
# extract the data we want
data = extract_player_data(table_rows)
#column_headers
# and then store it in a DataFrame
#df_2016 = pd.DataFrame(data, columns=column_headers)
df_2016  = pd.DataFrame(data)

In [20]:
data[10]

['2010',
 '1',
 '',
 'FrRnd',
 '1',
 'Nationals',
 'Y',
 'Bryce Harper\xa0(minors)',
 'OF',
 '21.5',
 '657',
 '2336',
 '121',
 '.279',
 '.883',
 '',
 '',
 '',
 '',
 '',
 '',
 'JC',
 'College of Southern Nevada (Henderson, NV)']

In [21]:
df_2016.columns = ['Year'] + column_headers
df_2016.head()

,Year,Rnd,DT,FrRnd,RdPck,Tm,Signed,Name,Pos,WAR,...,BA,OPS,G,W,L,ERA,WHIP,SV,Type,Drafted Out of
0,Type of School Drafted From:(not set for all s...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Name and Came From searches work only for name...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Picks by Position/Round=Pos=\nC\nP\nSS\nOF\n1B...,Player Name:,Came From:\n(enter 2-letter code for states)\n,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,2016,1,,FrRnd,1,Phillies,Y,Mickey Moniak (minors),OF,,...,,,,,,,,,HS,"La Costa Canyon HS (Carlsbad, CA)"


In [22]:
# Scraping all data up to the 1200th pick, creating DataFrame for each pick
draft_dfs_list = []
errors_list = []

In [23]:
url_template = "http://www.baseball-reference.com/draft/?overall_pick={pick}&draft_type=junreg&"

for pick in range (1, 1201):
    try: 
        url = url_template.format(pick = pick)
        html = urllib.request.urlopen(url)
        soup = BeautifulSoup(html, "lxml") 
        column_headers = [th.getText() for th in soup.findAll('th', limit=23)]
        column_headers = [s for s in column_headers if len(s) != 0]
        column_headers = column_headers[1:]
        
        # select the row for which I want to start collecting data
        table_rows = soup.find_all("tr")[5:]

        # extract the player data from the table rows
        player_data = extract_player_data(table_rows)
        
        pick_df = pd.DataFrame(player_data)
                        
        draft_dfs_list.append(pick_df)
        
    except Exception as e:
        
        error = [url, e]
        errors_list.append(error)

        
print(len((errors_list)))
errors_list

0


[]

In [24]:
draft_dfs_list[0:1]

[      0  1  2      3  4             5  6                           7    8   \
 0   2016  1     FrRnd  1      Phillies  Y      Mickey Moniak (minors)   OF   
 1   2015  1     FrRnd  1  Diamondbacks  Y     Dansby Swanson (minors)   SS   
 2   2014  1     FrRnd  1        Astros  N        Brady Aiken (minors)  LHP   
 3   2013  1     FrRnd  1        Astros  Y         Mark Appel (minors)  RHP   
 4   2012  1     FrRnd  1        Astros  Y      Carlos Correa (minors)   SS   
 5   2011  1     FrRnd  1       Pirates  Y        Gerrit Cole (minors)  RHP   
 6   2010  1     FrRnd  1     Nationals  Y       Bryce Harper (minors)   OF   
 7   2009  1     FrRnd  1     Nationals  Y  Stephen Strasburg (minors)  RHP   
 8   2008  1     FrRnd  1          Rays  Y        Tim Beckham (minors)   SS   
 9   2007  1     FrRnd  1    Devil Rays  Y        David Price (minors)  LHP   
 10  2006  1     FrRnd  1        Royals  Y      Luke Hochevar (minors)  RHP   
 11  2005  1     FrRnd  1  Diamondbacks  Y       Jus

In [25]:
column_headers.insert(0, "Year")
print(column_headers)


['Year', 'Rnd', 'DT', 'FrRnd', 'RdPck', 'Tm', 'Signed', 'Name', 'Pos', 'WAR', 'G', 'AB', 'HR', 'BA', 'OPS', 'G', 'W', 'L', 'ERA', 'WHIP', 'SV', 'Type', 'Drafted Out of']


In [1]:
draft_df = pd.concat(draft_dfs_list, axis = 0)

draft_df.columns = column_headers

draft_df.head()

NameError: name 'pd' is not defined

In [86]:
draft_df.loc[0,:]

,Year,Rnd,DT,FrRnd,RdPck,Tm,Signed,Name,Pos,WAR,...,BA,OPS,G,W,L,ERA,WHIP,SV,Type,Drafted Out of
0,2016,1,,FrRnd,1,Phillies,Y,Mickey Moniak (minors),OF,,...,,,,,,,,,HS,"La Costa Canyon HS (Carlsbad, CA)"
0,2016,1,,FrRnd,2,Reds,Y,Nick Senzel (minors),3B,,...,,,,,,,,,4Yr,"University of Tennessee (Knoxville, TN)"
0,2016,1,,FrRnd,3,Braves,Y,Ian Anderson (minors),RHP,,...,,,,,,,,,HS,"Shenendehowa HS (Ballston Lake, NY)"
0,2016,1,,FrRnd,4,Rockies,Y,Riley Pint (minors),RHP,,...,,,,,,,,,HS,"St. Thomas Aquinas HS (Overland Park, KS)"
0,2016,1,,FrRnd,5,Brewers,Y,Corey Ray (minors),OF,,...,,,,,,,,,4Yr,"University of Louisville (Louisville, KY)"
0,2016,1,,FrRnd,6,Athletics,Y,A.J. Puk (minors),LHP,,...,,,,,,,,,4Yr,"University of Florida (Gainesville, FL)"
0,2016,1,,FrRnd,7,Marlins,Y,Braxton Garrett (minors),LHP,,...,,,,,,,,,HS,"Florence HS (Florence, AL)"
0,2016,1,,FrRnd,8,Padres,Y,Cal Quantrill (minors),RHP,,...,,,,,,,,,4Yr,"Stanford University (Palo Alto, CA)"
0,2016,1,,FrRnd,9,Tigers,Y,Matt Manning (minors),RHP,,...,,,,,,,,,HS,"Sheldon HS (Sacramento, CA)"
0,2016,1,,FrRnd,10,White Sox,Y,Zack Collins (minors),C,,...,,,,,,,,,4Yr,"University of Miami (Coral Gables, FL)"


In [90]:
# save file
draft_df.to_csv("/Users/hsujohnathan/Google Drive/SOC 401/MLB Draft/data/raw_data/pfr_MLB_draft_data_RAW.csv", index = False)

# Cleaning the Data
Now that we have the raw passing data, we need to clean it up a bit in order to do some of the data exploration we want.

In [11]:
os.getcwd()

'/Users/hsujohnathan/Google Drive/SOC 401/MLB Draft/notebook'

In [3]:
draft_df_clean = pd.read_csv("../data/raw_data/pfr_MLB_draft_data_RAW.csv", encoding = "ISO-8859-1", low_memory = False)

In [4]:
draft_df_clean
draft_df_clean.dtypes

Year                int64
Rnd                object
DT                float64
FrRnd              object
RdPck             float64
Tm                 object
Signed             object
Name               object
Pos                object
WAR               float64
G                 float64
AB                float64
HR                float64
BA                float64
OPS               float64
G.1               float64
W                 float64
L                 float64
ERA               float64
WHIP              float64
SV                float64
Type               object
Drafted Out of     object
dtype: object

In [6]:
draft_df_clean.columns

Index(['Year', 'Rnd', 'DT', 'FrRnd', 'RdPck', 'Tm', 'Signed', 'Name', 'Pos',
       'WAR', 'G', 'AB', 'HR', 'BA', 'OPS', 'G.1', 'W', 'L', 'ERA', 'WHIP',
       'SV', 'Type', 'Drafted Out of'],
      dtype='object')

In [25]:
draft_df_clean["Drafted Out of"] = draft_df_clean["Drafted Out of"].astype(str)
draft_df_clean["State"] = draft_df_clean["Drafted Out of"].apply(lambda x: x[-3:].replace(")", ""))
draft_df_clean["State"].unique()

array(['AZ', 'CA', 'TX', 'FL', 'NV', 'OH', 'MO', 'MD', 'PA', 'MA', 'CT',
       'MN', 'OK', 'VA', 'AL', 'IL', 'MI', 'MS', 'PR', 'IN', 'NY', 'GA',
       'WA', 'NJ', 'CO', 'TN', 'LA', 'AR', 'SC', 'ON', 'OR', 'UT', 'KS',
       'KY', 'IA', 'QC', 'nan', 'NH', 'NC', 'HI', 'NE', 'ND', 'WI', 'BC',
       'MB', 'DE', 'WV', 'ME', 'ID', 'SD', 'VT', 'NM', 'RI', 'WY', 'DC',
       'AB', 'MT', 'NS', 'AK', 'SK', '(FL', 'VI', 'NL', 'NB'], dtype=object)

In [28]:
draft_columns = {'Year':'Year', 'Rnd':'Round','DT':'DT', 'FrRnd':'FrRnd', 'RdPck':'Round Pick',
                 'Tm':'Team', 'Signed':'Signed', 'Name':'Name', 'Pos':'Position',
                 'WAR':'WAR', 'G':'Games Played',
                 'AB': 'At bat', 'HR':'Homerun', 'BA':'BA',
                 'OPS': 'OPS', 'G.1':'Games Pitched', 'W': 'Wins',
                 'L':'Losses', 'ERA':'ERA', 'WHIP':'WHIP', 'SV':'Saves',
                 'Type': 'Draft Type', 'Drafted Out of': 'Drafted Out of'
                }
ordered_columns =  ['Year', 'Round',
                 'Team', 'Signed', 'Name', 'Position',
                 'WAR', 'Games Played',
                 'At bat', 'Homerun', 'BA',
                 'OPS', 'Wins',
                 'Losses', 'ERA', 'WHIP',
                 'Saves', 'Draft Type', 'State'
                ]

In [29]:
draft_df_clean.rename(index=str, columns = draft_columns, inplace = True)
draft_df_clean

,Year,Round,Team,Signed,Name,Position,WAR,Games Played,At bat,Homerun,BA,OPS,Wins,Losses,ERA,WHIP,Saves,Draft Type,Drafted Out of,State
291,1985,1,Pirates,Y,Barry BondsåÊ(minors),OF,162.4,2986.0,9847.0,762.0,0.298,1.051,0.0,0.0,0.00,0.00,0.0,4Yr,"Arizona State University (Tempe, AZ)",AZ
2014,1982,2,Giants,N,Barry BondsåÊ(minors),OF,162.4,2986.0,9847.0,762.0,0.298,1.051,0.0,0.0,0.00,0.00,0.0,HS,"Serra HS (San Mateo, CA)",CA
15025,1981,12,Mets,N,Roger ClemensåÊ(minors),RHP,140.3,709.0,179.0,0.0,0.173,0.443,354.0,184.0,3.12,1.17,0.0,JC,"San Jacinto College (Pasadena, TX)",TX
969,1983,1,Red Sox,Y,Roger ClemensåÊ(minors),RHP,140.3,709.0,179.0,0.0,0.173,0.443,354.0,184.0,3.12,1.17,0.0,4Yr,"University of Texas at Austin (Austin, TX)",TX
23,1993,1,Mariners,Y,Alex RodriguezåÊ(minors),SS,117.7,2784.0,10566.0,696.0,0.295,0.930,0.0,0.0,0.00,0.00,0.0,HS,"Westminster Christian (Miami, FL)",FL
4987,1976,4,Athletics,Y,Rickey HendersonåÊ(minors),OF,110.8,3081.0,10961.0,297.0,0.279,0.820,0.0,0.0,0.00,0.00,0.0,HS,"Technical HS (Oakland, CA)",CA
10047,1965,10,Dodgers,N,Tom SeaveråÊ(minors),RHP,110.5,677.0,1315.0,12.0,0.154,0.429,311.0,205.0,2.86,1.12,1.0,4Yr,University of Southern California (Los Angeles...,CA
1595,1984,2,Cubs,Y,Greg MadduxåÊ(minors),RHP,106.9,759.0,1591.0,5.0,0.171,0.395,355.0,227.0,3.16,1.14,0.0,HS,"Valley HS (Las Vegas, NV)",NV
1556,1971,2,Phillies,Y,Mike SchmidtåÊ(minors),SS,106.5,2404.0,8352.0,548.0,0.267,0.908,0.0,0.0,0.00,0.00,0.0,4Yr,"Ohio University (Athens, OH)",OH
4616,1982,4,Braves,N,Randy JohnsonåÊ(minors),LHP,102.1,619.0,625.0,1.0,0.125,0.305,303.0,166.0,3.29,1.17,2.0,HS,"Livermore HS (Livermore, CA)",CA


In [30]:
draft_df_clean.head(10)

,Year,Round,Team,Signed,Name,Position,WAR,Games Played,At bat,Homerun,BA,OPS,Wins,Losses,ERA,WHIP,Saves,Draft Type,Drafted Out of,State
291,1985,1,Pirates,Y,Barry BondsåÊ(minors),OF,162.4,2986.0,9847.0,762.0,0.298,1.051,0.0,0.0,0.00,0.00,0.0,4Yr,"Arizona State University (Tempe, AZ)",AZ
2014,1982,2,Giants,N,Barry BondsåÊ(minors),OF,162.4,2986.0,9847.0,762.0,0.298,1.051,0.0,0.0,0.00,0.00,0.0,HS,"Serra HS (San Mateo, CA)",CA
15025,1981,12,Mets,N,Roger ClemensåÊ(minors),RHP,140.3,709.0,179.0,0.0,0.173,0.443,354.0,184.0,3.12,1.17,0.0,JC,"San Jacinto College (Pasadena, TX)",TX
969,1983,1,Red Sox,Y,Roger ClemensåÊ(minors),RHP,140.3,709.0,179.0,0.0,0.173,0.443,354.0,184.0,3.12,1.17,0.0,4Yr,"University of Texas at Austin (Austin, TX)",TX
23,1993,1,Mariners,Y,Alex RodriguezåÊ(minors),SS,117.7,2784.0,10566.0,696.0,0.295,0.930,0.0,0.0,0.00,0.00,0.0,HS,"Westminster Christian (Miami, FL)",FL
4987,1976,4,Athletics,Y,Rickey HendersonåÊ(minors),OF,110.8,3081.0,10961.0,297.0,0.279,0.820,0.0,0.0,0.00,0.00,0.0,HS,"Technical HS (Oakland, CA)",CA
10047,1965,10,Dodgers,N,Tom SeaveråÊ(minors),RHP,110.5,677.0,1315.0,12.0,0.154,0.429,311.0,205.0,2.86,1.12,1.0,4Yr,University of Southern California (Los Angeles...,CA
1595,1984,2,Cubs,Y,Greg MadduxåÊ(minors),RHP,106.9,759.0,1591.0,5.0,0.171,0.395,355.0,227.0,3.16,1.14,0.0,HS,"Valley HS (Las Vegas, NV)",NV
1556,1971,2,Phillies,Y,Mike SchmidtåÊ(minors),SS,106.5,2404.0,8352.0,548.0,0.267,0.908,0.0,0.0,0.00,0.00,0.0,4Yr,"Ohio University (Athens, OH)",OH
4616,1982,4,Braves,N,Randy JohnsonåÊ(minors),LHP,102.1,619.0,625.0,1.0,0.125,0.305,303.0,166.0,3.29,1.17,2.0,HS,"Livermore HS (Livermore, CA)",CA


In [31]:
draft_df_clean = pd.DataFrame(data = draft_df_clean, columns = ordered_columns)
draft_df_clean = draft_df_clean.apply(pd.to_numeric, errors="ignore")

In [32]:
num_cols = draft_df_clean.columns[draft_df_clean.dtypes != object]
draft_df_clean.loc[:, num_cols] = draft_df_clean.loc[:, num_cols].fillna(0)

In [33]:
draft_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54308 entries, 291 to 4783
Data columns (total 19 columns):
Year            54308 non-null int64
Round           54308 non-null object
Team            54308 non-null object
Signed          54308 non-null object
Name            54308 non-null object
Position        54301 non-null object
WAR             54308 non-null float64
Games Played    54308 non-null float64
At bat          54308 non-null float64
Homerun         54308 non-null float64
BA              54308 non-null float64
OPS             54308 non-null float64
Wins            54308 non-null float64
Losses          54308 non-null float64
ERA             54308 non-null float64
WHIP            54308 non-null float64
Saves           54308 non-null float64
Draft Type      53822 non-null object
State           54308 non-null object
dtypes: float64(11), int64(1), object(7)
memory usage: 8.3+ MB


In [34]:
draft_df_clean = draft_df_clean[pd.notnull(draft_df_clean['Name'])]
draft_df_clean.sort_values('WAR', ascending=False, axis=0, inplace=True)

In [35]:
draft_df_clean.to_csv("/Users/hsujohnathan/Google Drive/SOC 401/MLB Draft/data/clean_data/pfr_MLB_draft_data_clean_python.csv", index=False)
draft_df_clean.head(10)

,Year,Round,Team,Signed,Name,Position,WAR,Games Played,At bat,Homerun,BA,OPS,Wins,Losses,ERA,WHIP,Saves,Draft Type,State
291,1985,1,Pirates,Y,Barry BondsåÊ(minors),OF,162.4,2986.0,9847.0,762.0,0.298,1.051,0.0,0.0,0.00,0.00,0.0,4Yr,AZ
2014,1982,2,Giants,N,Barry BondsåÊ(minors),OF,162.4,2986.0,9847.0,762.0,0.298,1.051,0.0,0.0,0.00,0.00,0.0,HS,CA
15025,1981,12,Mets,N,Roger ClemensåÊ(minors),RHP,140.3,709.0,179.0,0.0,0.173,0.443,354.0,184.0,3.12,1.17,0.0,JC,TX
969,1983,1,Red Sox,Y,Roger ClemensåÊ(minors),RHP,140.3,709.0,179.0,0.0,0.173,0.443,354.0,184.0,3.12,1.17,0.0,4Yr,TX
23,1993,1,Mariners,Y,Alex RodriguezåÊ(minors),SS,117.7,2784.0,10566.0,696.0,0.295,0.930,0.0,0.0,0.00,0.00,0.0,HS,FL
4987,1976,4,Athletics,Y,Rickey HendersonåÊ(minors),OF,110.8,3081.0,10961.0,297.0,0.279,0.820,0.0,0.0,0.00,0.00,0.0,HS,CA
10047,1965,10,Dodgers,N,Tom SeaveråÊ(minors),RHP,110.5,677.0,1315.0,12.0,0.154,0.429,311.0,205.0,2.86,1.12,1.0,4Yr,CA
1595,1984,2,Cubs,Y,Greg MadduxåÊ(minors),RHP,106.9,759.0,1591.0,5.0,0.171,0.395,355.0,227.0,3.16,1.14,0.0,HS,NV
1556,1971,2,Phillies,Y,Mike SchmidtåÊ(minors),SS,106.5,2404.0,8352.0,548.0,0.267,0.908,0.0,0.0,0.00,0.00,0.0,4Yr,OH
4616,1982,4,Braves,N,Randy JohnsonåÊ(minors),LHP,102.1,619.0,625.0,1.0,0.125,0.305,303.0,166.0,3.29,1.17,2.0,HS,CA
